Beata Purvina

# SQL Query Optimization Tutorial

In this tutorial we'll explore different ways to optimize SQL database queries using indexes, views, temporary tables, and subqueries. We will be using the [sqlite3 library](https://docs.python.org/2/library/sqlite3.html) to perform queries on our database and a [function](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_sql_query.html) from pandas library to see the output from our queries. We'll be connecting directly to the chinook.db database file provided by [SQLite](https://www.sqlitetutorial.net/sqlite-sample-database/). This tutorial assumes basic knowledge of SQL.

In practice, database queries are finetuned to run efficiently based on each query's unique inefficiency. These vary widely based on how often the database is updated, how often the query is performed, and whether the user prefers faster read operations or faster write operations, amongst other factors.

For the purposes of this tutorial, we'll be working on a relatively small dataset so the time differences between efficient and less efficient queries will be small, if at all evident. The goal is to become familiar with the common tools that can be used to speed up most queries on large datasets. As tables in the database grow large, smallest adjustments to queries can make significant execution time differences.

Throughout this tutorial it will be useful to refer to the following chinook database diagram which lists all tables and their relations. Table names are written as orange headers and column names are listed under each table header. Yellow keys mark the primary keys of each table.

![Chinook database diagram](database.jpg)

Things we'll cover:

0. [Set Up](#Part-0:-Set-Up)
1. [Indexes](#Part-1:-Indexes)
2. [Views](#Part-2:-Views)
3. [Temporary Tables](#Part-3:-Temporary-Tables)
4. [Subqueries](#Part-4:-Subqueries)
5. [Additional Resources](#Additional-Resources)


## Part 0: Set Up

We'll first connect to the chinook.db database file stored in the same directory as the tutorial and define a helper funtion that will execute our queries in chinook and output the time each query took to compute. An additional copy of chinook.db as it is provided by SQLite is stored in the _Clean_ folder and can also be downloaded [here](https://www.sqlitetutorial.net/sqlite-sample-database/) so indexes, views and temporary tables can be recreated again.

In [1]:
import sqlite3
import pandas as pd
import time

In [2]:
# Connect to the chinook database
conn = sqlite3.connect("chinook.db")
c = conn.cursor()

In [3]:
# Helper function
def time_to_execute(sql):
    start = time.time()
    c.execute(sql)
    print('Time to execute: %.4f sec' %(time.time() - start))

## Part 1: Indexes

Indexes are often the first go-to mechanism for speeding up queries. They allow faster row lookup for a given column name. Instead of having to iterate through every row in a table, the program quckly determines the position of the needed row based on a given indexed column value. Primary keys in a table are indexed by default, but additional indexes based on one or more columns and column expressions can be created manually. In the chinkook database that we'll be using all primary keys and foreign keys are indexed.

As an example, suppose we would like to see all tracks in our database that are between 2 and 5 minutes in length. We'd like to list each such track's name and length in minutes. 

For our own reference, let's first see what we're working with by computing the total number of tracks and the lengths of the longest and shortest tracks.

In [4]:
# Total number of tracks: 3503
c.execute("""SELECT COUNT(*) FROM tracks""")
c.fetchall()

[(3503,)]

In [5]:
# Length of longest track: ~88 minutes
# Length of shorest track: ~1 second
c.execute("""SELECT MAX(Milliseconds)/60000, MIN(Milliseconds)/1000 FROM tracks""")
c.fetchall()

[(88, 1)]

Let's first find all tracks that are between 2 and 5 minutes in length without creating any additional indexes. Remember that _TrackID_ is the primary key of the tracks table so it is indexed by default and foreign keys _AlbumId, MediaTypeId_ and _GenreId_ are indexed, as well.

In [6]:
track_len = """SELECT Name, (Milliseconds/60000) as Length 
            FROM tracks 
            WHERE (Milliseconds/60000) BETWEEN 2 AND 5
            ORDER BY Milliseconds DESC"""

time_to_execute(track_len)

Time to execute: 0.0049 sec


In [7]:
# To see output
pd.read_sql_query(track_len, conn)

,Name,Length
0,2 Minutes To Midnight,5
1,Carolina Hard-Core Ecstasy,5
2,Loosen My Strings,5
3,Too Many Tears,5
4,Black Moon Creeping,5
...,...,...
2782,Papeau Nuky Doe,2
2783,A Different Kind Of Blue,2
2784,She's A Rebel,2
2785,"Aria Mit 30 Veränderungen, BWV 988 ""Goldberg V...",2


To see how the query is computed we can use the _EXPLAIN_ command.

In [8]:
# See how query will be executed
pd.read_sql_query("EXPLAIN QUERY PLAN " + track_len, conn)

,id,parent,notused,detail
0,3,0,0,SCAN TABLE tracks
1,16,0,0,USE TEMP B-TREE FOR ORDER BY


Based on the _EXPLAIN_ output, we see that the program iteratevely scanned the _tracks_ table to find the values of _Milliseconds_ column that satisfy our condition.

Now let's create an index named _track_len_min_ for the _Milliseconds_ column since we are using that column in our _WHERE_ condition and run our query again. Note that we'll need to create an index based on the exact expression _Milliseconds/60000_ that we use in our _WHERE_ clause. Otherwise the index will not be used.

In [9]:
create_len_idx = """
        CREATE INDEX IF NOT EXISTS track_len_min 
        ON tracks(Milliseconds/60000)"""

c.execute(create_len_idx)
conn.commit()

In [10]:
time_to_execute(track_len)

Time to execute: 0.0134 sec


In [11]:
# See how query will be executed
pd.read_sql_query("EXPLAIN QUERY PLAN " + track_len, conn)

,id,parent,notused,detail
0,4,0,0,SEARCH TABLE tracks USING INDEX track_len_min ...
1,16,0,0,USE TEMP B-TREE FOR ORDER BY


After running our query again, this time on an indexed condition, we see in the _EXPLAIN_ output that the index _track_len_min_ was indeed used to compute our query. The underlying algorithm no longer needed to scan every row in our table to find the _Milliseconds_ entries that matched our condition and instead was able to perform a quicker search. While scanning the table does about $O(n)$ work, search on an indexed column does $O(\log n)$ work.

However, since our table is relatively small, the overhead costs of using indexes may outweigh the time gains. In practice, we would refrain from using indexes on small tables.

In order to drop an index, we can use the following command.

In [12]:
c.execute("""DROP INDEX IF EXISTS track_len_min""")
conn.commit()

## Part 2: Views

In this part of our tutorial we'll explore a way to write an efficient a query that's based on combining data from multiple tables. Views are table-like structures that can be created from a query, stored in a database and queried. Views are often used for de-normalizing data and cutomizing outputs for specific users. De-normalizing data essentially means combining multiple entities into a single table. In a proper database design, each table should store data about a single entity to avoid data redundancy which leads to [anomalies](https://databasemanagement.fandom.com/wiki/Category:Data_Anomalies). However, queries on de-normalized data are often much more time efficient. Once a view is created, it is visible to multiple clients using the database, unlike temporary tables (that we'll cover in part 3), which are session-specific.

Suppose we would like to compute the top 5 best-selling albums for a specific year. The most intuitive way to do this would be joining 5 tables and specifying the year of interest.

In [13]:
top5_albums = """
            SELECT albums.Title AS Album, artists.Name AS Artist,
                SUM(invoice_items.Quantity*invoice_items.UnitPrice) AS Value,
                SUM(invoice_items.Quantity) AS Quantity
            FROM albums INNER JOIN artists
                ON albums.ArtistId = artists.ArtistId
                INNER JOIN tracks
                ON albums.AlbumId = tracks.AlbumId
                INNER JOIN invoice_items
                ON tracks.TrackId = invoice_items.TrackId
                INNER JOIN invoices
                ON invoice_items.InvoiceId = invoices.InvoiceId
            WHERE STRFTIME('%Y', invoices.InvoiceDate) = '2010'
            GROUP BY Album
            ORDER BY Value DESC
            LIMIT 5;"""

time_to_execute(top5_albums)

Time to execute: 0.0080 sec


In [14]:
# To see output
pd.read_sql_query(top5_albums, conn)

,Album,Artist,Value,Quantity
0,Acústico,Titãs,8.91,9
1,Greatest Kiss,Kiss,8.91,9
2,Prenda Minha,Caetano Veloso,8.91,9
3,"Battlestar Galactica (Classic), Season 1",Battlestar Galactica (Classic),7.96,4
4,"Battlestar Galactica, Season 3",Battlestar Galactica,7.96,4


However, _JOIN_ operations are expensive, especially if we need to run this query multiple times. If the data is not updated frequently and a specific set of operations is frequently performed - in our case finding top 5 best-selling albums over a given time period - it may be worthwhile to create a de-normalized view of the data. We'll execute the join operations once and will query the view directly. This view will include the following columns:

Album | Artist | Track | QuantitySold | ValueSold | DateSold

In [15]:
# Create a view albumSales

create_view = """
            CREATE VIEW IF NOT EXISTS albumSales AS
            SELECT albums.Title AS Album, artists.Name AS Artists,
                tracks.Name AS Track, invoice_items.Quantity,
                (invoice_items.Quantity*invoice_items.UnitPrice) AS Value,
                invoices.InvoiceDate AS DateSold
            FROM albums INNER JOIN artists
                ON albums.ArtistId = artists.ArtistId
                INNER JOIN tracks
                ON albums.AlbumId = tracks.AlbumId
                INNER JOIN invoice_items
                ON tracks.TrackId = invoice_items.TrackId
                INNER JOIN invoices
                ON invoice_items.InvoiceId = invoices.InvoiceId;
            """
time_to_execute(create_view)
conn.commit()

Time to execute: 0.0025 sec


In [16]:
# To see the first few entries of our view
pd.read_sql_query("SELECT * FROM albumSales LIMIT 5;",conn)

,Album,Artists,Track,Quantity,Value,DateSold
0,Balls to the Wall,Accept,Balls to the Wall,1,0.99,2009-01-01 00:00:00
1,Restless and Wild,Accept,Restless and Wild,1,0.99,2009-01-01 00:00:00
2,For Those About To Rock We Salute You,AC/DC,Put The Finger On You,1,0.99,2009-01-02 00:00:00
3,For Those About To Rock We Salute You,AC/DC,Inject The Venom,1,0.99,2009-01-02 00:00:00
4,For Those About To Rock We Salute You,AC/DC,Evil Walks,1,0.99,2009-01-02 00:00:00


Note that we could also group the _albumSales_ view by year. That will speed up future queries even further, but will limit our flexibility in case we'd like to see album sales for a specific month or day rather than a year.

Now, getting the top 5 best-selling albums in the year 2010 is quicker. The time difference is much more noticeable in a large table.

In [17]:
top5_albums_view = """
            SELECT Album, Artists, SUM(Value) AS Value, SUM(Quantity) AS Quantity
            FROM albumSales
            WHERE STRFTIME('%Y', DateSold) = '2010'
            GROUP BY Album
            ORDER BY Value DESC
            LIMIT 5;"""

time_to_execute(top5_albums_view)

Time to execute: 0.0039 sec


In [18]:
# To see output
pd.read_sql_query(top5_albums_view, conn)

,Album,Artists,Value,Quantity
0,Acústico,Titãs,8.91,9
1,Greatest Kiss,Kiss,8.91,9
2,Prenda Minha,Caetano Veloso,8.91,9
3,"Battlestar Galactica (Classic), Season 1",Battlestar Galactica (Classic),7.96,4
4,"Battlestar Galactica, Season 3",Battlestar Galactica,7.96,4


## Part 3: Temporary Tables

Temporary tables are similar to views in that they can also be created by querying existing tables and can be queried themselves. Unlike views, temporary tables are only visible to the user that created them and are automatically dropped once the connection to the database is closed. 

Suppose we wanted to compute query A and query B, both of which will need to use output from query C. Rather than computing C twice, separately for A and B, we can compute C and store it in a temporary table for quick access.

In this example we'll create a temporary table storing (C) each customer's spending to be used as an intermediary computation in determining (A) the total value of all purchases serviced by each employee and (B) the top 3 spending countries. Our temporary table will have the following columns:

CustomerId | TotalSpent | Country | SupportRepId

In [19]:
# Create temporary table (C)
create_temp_table = """
            CREATE TEMPORARY TABLE IF NOT EXISTS custSpending AS
            SELECT customers.CustomerId, SUM(invoices.Total) AS TotalSpent,
                customers.Country, customers.SupportRepId
            FROM invoices INNER JOIN customers
                ON invoices.CustomerId = customers.CustomerId
            GROUP BY customers.CustomerID;"""

time_to_execute(create_temp_table)
conn.commit()

Time to execute: 0.0020 sec


In [20]:
# To see the first few entries of our temporary table
pd.read_sql_query("SELECT * FROM custSpending LIMIT 5;",conn)

,CustomerId,TotalSpent,Country,SupportRepId
0,1,39.62,Brazil,3
1,2,37.62,Germany,5
2,3,39.62,Canada,3
3,4,39.62,Norway,4
4,5,40.62,Czech Republic,4


In [21]:
# Compute the total value of all purchases serviced by each employee (A)
empl_value = """
            SELECT (employees.FirstName || ' ' || employees.LastName) AS Employee,
                SUM(custSpending.TotalSpent) AS AmountServiced
            FROM employees LEFT OUTER JOIN custSpending
                ON employees.EmployeeId = custSpending.SupportRepId 
            GROUP BY employees.EmployeeId"""

time_to_execute(empl_value)

Time to execute: 0.0009 sec


In [22]:
# To see output
pd.read_sql_query(empl_value, conn)

,Employee,AmountServiced
0,Andrew Adams,NaN
1,Nancy Edwards,NaN
2,Jane Peacock,833.04
3,Margaret Park,775.40
4,Steve Johnson,720.16
5,Michael Mitchell,NaN
6,Robert King,NaN
7,Laura Callahan,NaN


Based on this output we can see that there are only three employees that have taken customer orders. NaN values mean there are no data entries.

In [23]:
# Compute top 3 spending countries (B)
top3_countries = """
            SELECT Country, (SUM(TotalSpent) || '$') AS Spending
            FROM custSpending
            GROUP BY Country
            ORDER BY SUM(TotalSpent) DESC
            LIMIT 3;"""

time_to_execute(top3_countries)

Time to execute: 0.0008 sec


In [24]:
# To see output
pd.read_sql_query(top3_countries, conn)

,Country,Spending
0,USA,523.06$
1,Canada,303.96$
2,France,195.1$


Not surprisingly, USA spent the most money on this service provider.

Note that the two queries above were simplified significantly and we avoided redundant computations by using a temporary table. Once we close our connection to the database, this table will be automatically dropped and space that it was taking up will be restored.

## Part 4: Subqueries

Subqueries are queries computed within queries. The most frequent places to call a subquery are within _FROM, WHERE_ and _HAVING_ statements. Subqueries can be correlated and non-correlated. Non-correlated subqueries are much more time efficient as they do not depend on the outer query and are therefore computed only once. Correlated queries are computed once for each row in the outer query that they depend on.

Suppose we would like to determine the number of movies stored in our database. To determine how to define a track as a movie, let's see all of the available tack genres.

In [25]:
pd.read_sql_query("SELECT Name as Genre FROM genres", conn)

,Genre
0,Rock
1,Jazz
2,Metal
3,Alternative & Punk
4,Rock And Roll
5,Blues
6,Latin
7,Reggae
8,Pop
9,Soundtrack


Based on this output we'll assume that a track is a movie if it is under "Sci Fi & Fantasy", "Drama" or "Comedy" genres.

An intuitive but perhaps inefficient way to compute the number of all movies is to join tables _genres_ and _tracks_ as follows.

In [26]:
tracks_in_genres = """
            SELECT COUNT(tracks.TrackId) AS "Number of Movies"
            FROM genres INNER JOIN tracks
                ON genres.GenreId = tracks.GenreId
            WHERE genres.Name IN ("Sci Fi & Fantasy", "Drama", "Comedy")"""

time_to_execute(empl_value)

Time to execute: 0.0001 sec


In [27]:
# To see output
pd.read_sql_query(tracks_in_genres, conn)

,Number of Movies
0,107


If the columns on which we're joining tables aren't indexed, the _JOIN_ operation will scan the right table for every left join key in the left table, doing approximately $O(n^2)$ work. In this case _genre.GenreId_ and _tracks.GenreId_ are primary and foreign keys, respectively, so the _JOIN_ operation will be performed quicker.

Using a _JOIN_ in this case is still wasteful as we don't need to join the _tracks_ table with any genres that aren't Sci Fi & Fantasy, Drama or Comedy. This is especially evident when the tables we are joining are large. To avoid unnecessary computations, we can use a subquery.

In [28]:
tracks_in_genres_sub = """
            SELECT COUNT(TrackId) AS "Number of Movies"
            FROM tracks
            WHERE GenreId IN (SELECT GenreId FROM genres WHERE Name IN ("Sci Fi & Fantasy", "Drama", "Comedy"))"""

time_to_execute(empl_value)

Time to execute: 0.0001 sec


In [29]:
# To see output
pd.read_sql_query(tracks_in_genres_sub, conn)

,Number of Movies
0,107


In this example we were able to avoid unnecessary computations and simplify our query. Our subquery generated a result set consisting of only three values, which we then used directly in our WHERE clause.

In [30]:
conn.close()

## Additional Resources

- More on [indexes](https://www.tutorialspoint.com/sql/sql-indexes.htm)
- More on [views](https://www.w3schools.com/sql/sql_view.asp) 
- More on [temporary tables](https://codingsight.com/introduction-to-temporary-tables-in-sql-server/)
- More on [subqueries](https://www.geeksforgeeks.org/sql-sub-queries/)